In [1]:
import sys
sys.path.insert(1, "./rnn")
import train
import model


In [2]:
from FileReader import FileReader
classInstance = FileReader
df = classInstance.get_dataframe() #IMPORTANT:  saves a pickle to output/simple or output/full. 
import sys
sys.path.insert(1, "./feature-extraction/embed-extractor")
#from EmbedExtractor import EmbedExtractor
sys.path.insert(1, "./feature-extraction/vulgar-extractor")
from VulgarExtractor import VulgarExtractor
sys.path.insert(1, "./feature-extraction/twitter-parser")
from TwitterParser import TwitterParser
import classifiers
import pandas as pd
import numpy as np
import json
#ee = EmbedExtractor()





 553480082996879360

 581153923987206146

 500298588992593920

 552788945017516032

 524948206023880704

 553561170637238272

 758159624122097664

 767725956706414592

 580352273001410560

 581359544682614784

 763098277986209792

 581290271997968384

 775057555865206784

 500280249629036544

 498293668655423488

 774991078265094144

 553553288625672192

 524961721744900097

 524941720249978880

 544294893146091520

 768859780240773121

 764927075522260992

 544274544174071809

 498486826269548545

 544315472075042818

 524923293711998976

 769988636754505729

 524936793633083394

 553480082996879360

 581153923987206146

 500298588992593920

 552788945017516032

 524948206023880704

 553561170637238272

 758159624122097664

 767725956706414592

 580352273001410560

 581359544682614784

 763098277986209792

 581290271997968384

 775057555865206784

 500280249629036544

 498293668655423488

 774991078265094144

 553553288625672192

 524961721744900097

 524941720249978880

 54429489314

In [3]:
# need stuff from main.py to test transformation into Tensors

def normalize(column_name, df):
    std = df[column_name].std()
    norm_col = df[column_name].apply(lambda x: x - std)
    df[column_name] = norm_col

# builds the labels and vectorizations of given data
#if you want to fool around with including/excluding certain features and whatnot, this is the place to do it

def labels_and_vectors(file, index=0):
    df = pd.read_pickle(file)
    
    wordlist = VulgarExtractor.vulgarWords("feature-extraction/vulgar-extractor/badwords.txt") 
    dftext = df[['text']]
    result = dftext.applymap(lambda x: VulgarExtractor.containsVulgar(x,wordlist))
    df['isVulgar'] = result

    #word_embeddings = [ee.tweetVec(tagged_line) for tagged_line in df['text']]
    textlist = [txt.replace('\n','') for txt in df['text'].tolist()]
    tagged_sents = TwitterParser.tag(textlist)
    df['POS'] = tagged_sents

    processed_sents = []
    for tagged_sent in df['POS']:
        processed_words = []
        for word, tag in tagged_sent:
            if tag == 'U':
                processed_words.append('someurl')
            elif tag == '@':
                processed_words.append('@someuser')
            else:
                processed_words.append(word)
        sent = ' '.join(processed_words)
        processed_sents.append(sent)
    df['text'] = processed_sents

    word_counts = [TwitterParser.word_count(tagged_line) for tagged_line in df['POS']]
    pos_count_list = [TwitterParser.pos_counts(tagged_line) for tagged_line in df['POS']]
    contains_adjs = [TwitterParser.contains_adjectives(tagged_line) for tagged_line in df['POS']]
    contains_urls = [TwitterParser.contains_url(tagged_line) for tagged_line in df['POS']]
    contains_emojis = [TwitterParser.contains_emoji(tagged_line) for tagged_line in df['POS']]
    contains_abbrevs = [TwitterParser.contains_abbreviation(tagged_line) for tagged_line in df['POS']]

    df['wordCount'] = word_counts
    df['posCounts'] = pos_count_list
    df['containsAdjective'] = contains_adjs
    df['containsURL'] = contains_urls
    df['containsEmoji'] = contains_emojis
    df['containsAbbreviation'] = contains_abbrevs
    #df['wordEmbedding'] = word_embeddings


    for i, tag in enumerate(TwitterParser.tagset):
        tag_counts = []
        for pos_counts in df['posCounts']:
            tag_counts.append(pos_counts[i])
        column_name = 'num_' + tag
        df[column_name] = tag_counts
        normalize(column_name, df)
        
    # Changes "true"/"false"/"unverified" to numeric values, just like the in the early cells

    df.loc[df.classification == 'true', 'classification'] = 1
    df.loc[df.classification == 'false', 'classification'] = -1
    df.loc[df.classification == 'unverified', 'classification'] =0
    # getting the labels

    #removed containsURL
    attributes = ['isVulgar', 'containsAdjective', 'containsEmoji', 'containsAbbreviation', 'wordCount']
    for tag in TwitterParser.tagset:
        attributes.append('num_' + tag)

    labels = df['classification']
    labels = [l for l in labels]
    labels = np.array(labels)


    # getting the values as a list of lists
    values = df[attributes].values.tolist()
    #word_embedding_values = df['wordEmbedding'].values.tolist()


    #Below puts the tweet ID as a feature. Comment this out if you aren't using tweetID
    ###for i,index in enumerate(df.index):
    ###    dev_values[i].append(int(index))


# UNCOMMENT THIS IN ORDER TO INCOPORATE WORD_EMBEDDINGS AGAIN
    #for i,d in enumerate(word_embedding_values):
     #   values[i].extend(d)

    values = np.array(values)
    if index == 1:
        return df.index, values

    return labels, values


In [4]:
tr_labels, tr_values = labels_and_vectors('output/simple/train_data_simple.pickle')
indices, dev_values,= labels_and_vectors('output/simple/dev_data_simple.pickle', index=1)

In [5]:
## convert to tensor Variable here
import tensorflow as tf
import torch
import torch.nn as nn
import helpers
from torch.autograd import Variable



In [9]:
n_epochs = 2000
print_every = 100
hidden_size = 50
learning_rate = 0.01
chunk_len = 200


decoder = model.RNN(len(tr_values[0]), hidden_size=hidden_size, output_size =3)
decoder_optimizer = torch.optim.SGD(decoder.parameters(), learning_rate)
# Loss function can be changed as an argument?

criterion = nn.NLLLoss()


In [7]:
tvs = [arr.tolist() for arr in tr_values]
tls = [l.item() for l in tr_labels]


In [10]:
#from the pytorch tutroial
def train_all():
    for i, label in enumerate(tls):
        category_tensor = Variable(torch.LongTensor([label]))
        line_tensor = Variable(torch.FloatTensor(tvs[i]))
        output, loss = train.train(category_tensor, line_tensor, decoder)
        current_loss += loss
        return category_tensor


# Print epoch number, loss, name and guess
# validation_loss = 0
# for key in category_lines_val.keys():
# lines = category_lines_val[key]
#         #total_count += len(lines)
#             for ln in lines:
#             output = evaluate(Variable(line_to_tensor(ln)))
#             category_tensor = Variable(torch.LongTensor([all_categories.index(key)]))
#             validation_loss += criterion(output, category_tensor)
#     validation_losses.append(validation_loss/900)

#     guess, guess_i = category_from_output(output)
#     correct = '✓' if guess == category else '✗ (%s)' % category
#     print('%d %d%% (%s) %.4f %s / %s %s' % (epoch, epoch / n_epochs * 100, time_since(start), loss, line, guess, correct))

lt = train_all()
#predictions = decoder.predict(dev_values)
    

#print(predictions)




0
Variable containing:
 0
[torch.LongTensor of size 1]

Variable containing:

Columns 0 to 9 
 0.1202  0.3391  0.0375  0.6061  0.4474  0.6548  0.6414  0.0466  0.3482  0.4437

Columns 10 to 19 
 0.7961  0.3999  0.4187  0.4269  0.9445  0.2572  0.8690  0.1079  0.2215  0.8181

Columns 20 to 29 
 0.0853  0.8358  0.1408  0.2640  0.2036  0.9084  0.2479  0.9250  0.1198  0.7834

Columns 30 to 39 
 0.1316  0.9842  0.4016  0.1913  0.6413  0.0993  0.2438  0.5629  0.6454  0.4048

Columns 40 to 49 
 0.6527  0.9045  0.3814  0.6932  0.7498  0.0428  0.6080  0.6607  0.6777  0.5242
[torch.FloatTensor of size 1x50]



RuntimeError: dimension out of range (expected to be in range of [-1, 0], but got 1)